# Lab 1: Introduction and Kubernetes-Based Deployment

## Overview

In this lab, you will:
- Set up your personal namespace in the shared Kubernetes cluster
- Deploy Dynamo using namespace-scoped operator on Kubernetes
- Configure a backend engine using aggregated serving
- Test the deployment with OpenAI-compatible API
- Benchmark the deployment using AI-Perf

## Duration: ~90 minutes

---

## Section 1: Environment Setup

### Objectives
- Verify Kubernetes access (shared cluster)
- Create your personal namespace
- Install Dynamo dependencies in your namespace
- Set up prerequisites (kubectl, helm)

### Prerequisites
Before starting, ensure you have:
- ✅ Kubernetes cluster access (kubeconfig provided by instructor)
- ✅ `kubectl` installed (version 1.24+) or `microk8s kubectl`
- ✅ `helm` 3.x installed
- ✅ HuggingFace token from [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)

### Step 2: Set Configuration Variables

Set your configuration variables. **Replace the values below with your own:**


In [7]:
import os

# Load environment variables from workshop-env.sh
# These are pre-configured based on your UID to prevent port conflicts
USER_JUPYTER_PORT = os.environ.get('USER_JUPYTER_PORT', '8888')
USER_FRONTEND_PORT = os.environ.get('USER_FRONTEND_PORT', '10000')
USER_FRONTEND2_PORT = os.environ.get('USER_FRONTEND2_PORT', '11000')
USER_PROMETHEUS_PORT = os.environ.get('USER_PROMETHEUS_PORT', '19090')
USER_GRAFANA_PORT = os.environ.get('USER_GRAFANA_PORT', '13000')
NAMESPACE = os.environ.get('NAMESPACE', f"dynamo-{os.environ.get('USER', 'unknown')}")

# Set workshop configuration
os.environ['RELEASE_VERSION'] = '0.6.0'
os.environ['NAMESPACE'] = NAMESPACE
os.environ['HF_TOKEN'] = ''  # Replace with your HuggingFace token
os.environ['CACHE_PATH'] = '/data/huggingface-cache'  # Shared cache path

print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print("🎓 Lab 1: Environment Configuration")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print(f"  User:             {os.environ.get('USER')}")
print(f"  Release Version:  {os.environ['RELEASE_VERSION']}")
print(f"  Namespace:        {NAMESPACE}")
print(f"  Cache Path:       {os.environ['CACHE_PATH']}")
print("")
print("📌 Your Assigned Ports:")
print(f"  Frontend (Lab 1): {USER_FRONTEND_PORT}")
print(f"  Prometheus:       {USER_PROMETHEUS_PORT}")
print(f"  Grafana:          {USER_GRAFANA_PORT}")
print("")
print("💡 Use localhost:{port} in your browser (via SSH tunnel)")
print("   - Frontend API:   localhost:10000")
print("   - Prometheus:     localhost:19090")
print("   - Grafana:        localhost:13000")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🎓 Lab 1: Environment Configuration
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  User:             nealv
  Release Version:  0.6.0
  Namespace:        dynamo-nealv
  Cache Path:       /data/huggingface-cache

📌 Your Assigned Ports:
  Frontend (Lab 1): 10030
  Prometheus:       19120
  Grafana:          13030

💡 Use localhost:{port} in your browser (via SSH tunnel)
   - Frontend API:   localhost:10000
   - Prometheus:     localhost:19090
   - Grafana:        localhost:13000
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


### Step 3: Verify Kubernetes Access


In [8]:
%%bash
# Verify kubectl is installed and configured
kubectl version --client

# Check cluster connection
kubectl cluster-info

# Check GPU nodes are available (optional)
kubectl get nodes -o custom-columns=NAME:.metadata.name,GPUs:.status.capacity.nvidia\\.com/gpu

Client Version: v1.34.1
Kustomize Version: v5.7.1


Kubernetes control plane is running at https://pu.mk8scluster-e00tennwzpd1k37qj0.mk8s.eu-north1.nebius.cloud:443
CoreDNS is running at https://pu.mk8scluster-e00tennwzpd1k37qj0.mk8s.eu-north1.nebius.cloud:443/api/v1/namespaces/kube-system/services/coredns:udp-53/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.
NAME                                 GPUs
computeinstance-e00chabkscp5qxrc2t   8
computeinstance-e00d922md5d0ygyyvp   8
computeinstance-e00f2xaap1zt5gff07   8
computeinstance-e00fc7ewzhrarns0kq   8
computeinstance-e00s38tvk7phv1d5h6   8
computeinstance-e00wg72jxq1qv0y88n   8
computeinstance-e00xmqnp8xy4n1xhtq   8


### Step 4: Create Your Personal Namespace


In [9]:
%%bash
# Create your personal namespace
kubectl create namespace $NAMESPACE

# Verify namespace was created
kubectl get namespace $NAMESPACE

Error from server (AlreadyExists): namespaces "dynamo-nealv" already exists


NAME           STATUS   AGE
dynamo-nealv   Active   106s


## Section 2: Install Dynamo Platform (Namespace-Scoped)

### Objectives
- Understand namespace-scoped operator deployment for shared clusters
- Install Dynamo CRDs (if not already installed cluster-wide)
- Install Dynamo platform (etcd, NATS, operator) in your namespace
- Verify platform components are running

### Architecture
```
Client → Frontend → Router → Worker(s) with Backend Engine
                        ↓
            etcd + NATS (namespace-scoped)
                        ↓
            Dynamo Operator (namespace-scoped)
```

### Important: Shared Cluster Configuration

Since we're using a **shared Kubernetes cluster**, each participant will:
- Create their own namespace (e.g., `dynamo-<yourname>`)
- Install a **namespace-scoped Dynamo operator** that only manages resources in your namespace
- The CRDs are cluster-wide and should already be installed (check first)

### Step 1: Check if Dynamo CRDs Are Installed

**Note:** CRDs are cluster-wide resources and only need to be installed **once per cluster**. If already installed, skip to Step 2.


In [10]:
%%bash
# Check if CRDs already exist
if kubectl get crd dynamographdeployments.nvidia.com &>/dev/null && \
   kubectl get crd dynamocomponentdeployments.nvidia.com &>/dev/null; then
    echo "✓ CRDs already installed"
    kubectl get crd | grep nvidia.com
else
    echo "⚠️  CRDs not found. Ask instructor to install them, or proceed with Step 1b"
fi

✓ CRDs already installed
clusterpolicies.nvidia.com                       2025-11-05T14:29:32Z
dynamocomponentdeployments.nvidia.com            2025-11-05T14:55:01Z
dynamographdeployments.nvidia.com                2025-11-05T14:55:02Z
nvidiadrivers.nvidia.com                         2025-11-05T14:29:32Z


### Step 1b: Install CRDs (Optional - Instructor May Do This)

**Skip this step if CRDs are already installed.** If needed, run:


In [ ]:
%%bash
# Install Dynamo CRDs (only if not already installed)
echo "Installing Dynamo CRDs..."
helm fetch https://helm.ngc.nvidia.com/nvidia/ai-dynamo/charts/dynamo-crds-$RELEASE_VERSION.tgz
helm install dynamo-crds dynamo-crds-$RELEASE_VERSION.tgz --namespace default

echo ""
echo "Verifying CRD installation:"
kubectl get crd | grep nvidia.com

### Step 2: Install Namespace-Scoped Dynamo Platform

This installs ETCD, NATS, and the Dynamo Operator Controller in your namespace with namespace restriction enabled.


In [12]:
%%bash
# Download platform chart
helm fetch https://helm.ngc.nvidia.com/nvidia/ai-dynamo/charts/dynamo-platform-$RELEASE_VERSION.tgz

# Install with namespace restriction enabled (IMPORTANT for shared clusters!)
echo "Installing Dynamo platform in namespace: $NAMESPACE"
helm install dynamo-platform dynamo-platform-$RELEASE_VERSION.tgz \
  --namespace $NAMESPACE \
  --set dynamo-operator.namespaceRestriction.enabled=true \
  --set metrics.enabled=false

echo ""
echo "Platform installation initiated. Waiting for pods to be ready..."

Installing Dynamo platform in namespace: dynamo-nealv
NAME: dynamo-platform
LAST DEPLOYED: Fri Nov  7 12:51:06 2025
NAMESPACE: dynamo-nealv
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
# SPDX-FileCopyrightText: Copyright (c) 2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
# SPDX-License-Identifier: Apache-2.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Platform installation initiated. Waiting for pods to be ready...


### Step 3: Wait for Platform Pods to Be Ready

Re-run the following cell until all pods report as "Running"


In [13]:
%%bash
kubectl get pods -n $NAMESPACE

NAME                                                              READY   STATUS    RESTARTS   AGE
dynamo-platform-dynamo-operator-controller-manager-64bbfc9mwkqw   2/2     Running   0          66s
dynamo-platform-etcd-0                                            1/1     Running   0          66s
dynamo-platform-nats-0                                            2/2     Running   0          66s


### Step 4: Create HuggingFace Token Secret


In [ ]:
%%bash
# Create HuggingFace token secret
kubectl create secret generic hf-token-secret \
  --from-literal=HF_TOKEN="" \
  --namespace $NAMESPACE

# Verify secret was created
kubectl get secret hf-token-secret -n $NAMESPACE
echo "✓ HuggingFace token secret created"

secret/hf-token-secret created
NAME              TYPE     DATA   AGE
hf-token-secret   Opaque   1      1s
✓ HuggingFace token secret created


## Section 3: Deploy Your First Model with Aggregated Serving

### Objectives
- Understand aggregated serving architecture
- Configure and deploy a model using vLLM backend
- Use Kubernetes manifests to deploy Dynamo resources

### Available Backends
In this lab, we'll use **vLLM** with aggregated serving for simplicity:
- **vLLM**: High-throughput serving with PagedAttention
- Model: `Qwen/Qwen2.5-1.5B-Instruct` (small, fast to download)
- Architecture: Aggregated serving with KV-cache routing

**Other backends** (for exploration):
- **SGLang**: Optimized for complex prompting and structured generation
- **TensorRT-LLM**: Maximum performance on NVIDIA GPUs

### Deployment Configuration

We'll use a `DynamoGraphDeployment` resource that defines:
- **Frontend**: OpenAI-compatible API endpoint with KV-cache routing
- **Worker**: vLLM worker with 1 GPU running the model

### Step 1: Update the Deployment Configuration

Before deploying, we need to update the YAML configuration with your specific values:


In [15]:
%%bash
# Update agg_router.yaml with your configuration

# Replace my-tag with actual version
sed -i "s/my-tag/$RELEASE_VERSION/g" agg_router.yaml

# Replace cache path
sed -i "s|/YOUR/LOCAL/CACHE/FOLDER|$CACHE_PATH|g" agg_router.yaml

echo "✓ Configuration updated in agg_router.yaml"
echo ""
echo "Verify image tags (should show version, not my-tag):"
grep "image:" agg_router.yaml

✓ Configuration updated in agg_router.yaml

Verify image tags (should show version, not my-tag):
          image: nvcr.io/nvidia/ai-dynamo/vllm-runtime:0.6.0
          image: nvcr.io/nvidia/ai-dynamo/vllm-runtime:0.6.0


### Step 2: Deploy the Model


In [17]:
%%bash
# Apply the deployment
kubectl apply -f agg_router.yaml --namespace $NAMESPACE

echo ""
echo "✓ Deployment created. This will take 4-6 minutes for first run."
echo "  - Pulling container images"
echo "  - Downloading model from HuggingFace"
echo "  - Loading model into GPU memory"

dynamographdeployment.nvidia.com/vllm-agg-router configured

✓ Deployment created. This will take 4-6 minutes for first run.
  - Pulling container images
  - Downloading model from HuggingFace
  - Loading model into GPU memory


### Step 3: Monitor Deployment Progress


In [18]:
%%bash
# Check deployment status
kubectl get dynamographdeployment -n $NAMESPACE

echo ""
echo "Pod status (wait for all pods to be 1/1 Ready):"
kubectl get pods -n $NAMESPACE | grep vllm

# To watch in real-time, uncomment the line below:
# kubectl get pods -n $NAMESPACE -w

NAME              AGE
vllm-agg-router   12m

Pod status (wait for all pods to be 1/1 Ready):
vllm-agg-router-frontend-6dfd867bd-6nc9n                          1/1     Running   0               12m
vllm-agg-router-vllmdecodeworker-76b79c4bf6-xdrdl                 1/1     Running   1 (3m46s ago)   4m30s


### Step 4: View Worker Logs (Optional)

While waiting for the deployment, you can watch the model loading progress:


In [19]:
%%bash
# Get logs from the worker pod
WORKER_POD=$(kubectl get pods -n $NAMESPACE | grep vllmdecodeworker | head -1 | awk '{print $1}')

if [ -n "$WORKER_POD" ]; then
    echo "Viewing logs from: $WORKER_POD"
    echo "Look for:"
    echo "  - 'Loading model weights...' (downloading)"
    echo "  - 'Model loading took X.XX GiB' (loaded)"
    echo ""
    kubectl logs $WORKER_POD -n $NAMESPACE --tail=30
else
    echo "Worker pod not found yet, please wait and try again"
fi

Viewing logs from: vllm-agg-router-vllmdecodeworker-76b79c4bf6-xdrdl
Look for:
  - 'Loading model weights...' (downloading)
  - 'Model loading took X.XX GiB' (loaded)

2025-11-07T18:05:34.388307Z  INFO kv_cache_utils.get_kv_cache_config_from_groups: Maximum concurrency for 32,768 tokens per request: 137.96x   
2025-11-07T18:05:34.406316Z  INFO nixl_connector.register_kv_caches: Registering KV_Caches. use_mla: False, kv_buffer_device: cuda, use_host_buffer: False   
2025-11-07T18:05:39.754020Z ERROR http-request: tower_http::trace::on_failure: response failed classification=Status code: 503 Service Unavailable latency=0 ms method=GET uri=/live version=HTTP/1.1
(EngineCore_DP0 pid=285) 2025-11-07 18:05:45,450 - INFO - autotuner.py:256 - flashinfer.jit: [Autotuner]: Autotuning process starts ...
(EngineCore_DP0 pid=285) 2025-11-07 18:05:45,486 - INFO - autotuner.py:262 - flashinfer.jit: [Autotuner]: Autotuning process ends
Capturing CUDA graphs (mixed prefill-decode, PIECEWISE):  94%|████

## Section 4: Testing and Validation

### Objectives
- Expose the service locally using port forwarding
- Send test requests to the deployment
- Verify OpenAI API compatibility
- Test streaming and non-streaming responses

### Testing Strategy
Once your deployment is running (`1/1 Ready`), you'll:
1. Forward the frontend service port to localhost
2. Test with curl commands
3. Verify response format and functionality

### Step 1: Set Up Port Forwarding

Forward the service port to localhost (run in background):


In [20]:
%%bash --bg
# Forward the service port (run in background)
kubectl port-forward deployment/vllm-agg-router-frontend $USER_FRONTEND_PORT:8000 -n $NAMESPACE &

echo "✓ Port forward started on localhost:${USER_FRONTEND_PORT}"
echo "  (To stop: use 'pkill -f port-forward' or press Ctrl+C)"
sleep 5  # Give it time to start

### Step 2: Test the `/v1/models` Endpoint


In [21]:
%%bash
curl http://localhost:${USER_FRONTEND_PORT}/v1/models

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   121  100   121    0     0    252      0 --:--:-- --:--:-- --:--:--   252    0     0    252      0 --:--:-- --:--:-- --:--:--   252


{"object":"list","data":[{"id":"Qwen/Qwen2.5-1.5B-Instruct","object":"object","created":1762538889,"owned_by":"nvidia"}]}

### Step 3: Simple Non-Streaming Chat Completion


In [23]:
!curl http://localhost:${USER_FRONTEND_PORT}/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{ \
    "model": "Qwen/Qwen2.5-1.5B-Instruct",\
    "messages": [{"role": "user", "content": "Hello! How are you?"}], \
    "stream": false,\
    "max_tokens": 50 \
  }'

╔════════════════════════════════════════════════════════════════╗
║          Dynamo Workshop - Multi-User Configuration            ║
╠════════════════════════════════════════════════════════════════╣
║ User: nealv (UID: 1030)
║ Namespace: dynamo-nealv
╠════════════════════════════════════════════════════════════════╣
║ Your assigned ports:
║   JupyterLab:        8918
║   Frontend (Lab 1):  10030
║   Frontend (Lab 2):  11030
║   Prometheus:        19120
║   Grafana:           13030
╠════════════════════════════════════════════════════════════════╣
║ SSH Tunnel Command (run on your local machine):
║   ssh -L 8888:localhost:8918 \
║       -L 10000:localhost:10030 \
║       -L 11000:localhost:11030 \
║       nealv@<workstation-hostname>
╚════════════════════════════════════════════════════════════════╝


{"id":"chatcmpl-4afce25d-fa58-451b-9e42-08c8403260ac","choices":[{"index":0,"message":{"content":"Hello! I'm just an artificial intelligence and don't have feelings or emotions like humans do. However, I'm here to help you with any questions or tasks you might have. Is there anything specific you'd like to talk about or learn more about","role":"assistant","reasoning_content":null},"finish_reason":"length"}],"created":1762538908,"model":"Qwen/Qwen2.5-1.5B-Instruct","object":"chat.completion","usage":null}

### Step 4: Test Streaming Response


In [24]:
!curl http://localhost:${USER_FRONTEND_PORT}/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{ \
    "model": "Qwen/Qwen2.5-1.5B-Instruct", \
    "messages": [{"role": "user", "content": "Write a short poem about AI"}], \
    "stream": true, \
    "max_tokens": 100 \
  }'

╔════════════════════════════════════════════════════════════════╗
║          Dynamo Workshop - Multi-User Configuration            ║
╠════════════════════════════════════════════════════════════════╣
║ User: nealv (UID: 1030)
║ Namespace: dynamo-nealv
╠════════════════════════════════════════════════════════════════╣
║ Your assigned ports:
║   JupyterLab:        8918
║   Frontend (Lab 1):  10030
║   Frontend (Lab 2):  11030
║   Prometheus:        19120
║   Grafana:           13030
╠════════════════════════════════════════════════════════════════╣
║ SSH Tunnel Command (run on your local machine):
║   ssh -L 8888:localhost:8918 \
║       -L 10000:localhost:10030 \
║       -L 11000:localhost:11030 \
║       nealv@<workstation-hostname>
╚════════════════════════════════════════════════════════════════╝
data: {"id":"chatcmpl-9f7008d5-e567-43ff-8716-1c9fdd57ed17","choices":[{"index":0,"delta":{"content":"AI","function_call":null,"tool_calls":null,"role":"assistant","refusal":null,"reasoning

### Step 5: Test with Different Parameters


In [25]:
!curl http://localhost:${USER_FRONTEND_PORT}/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{ \
    "model": "Qwen/Qwen2.5-1.5B-Instruct", \
    "messages": [{"role": "user", "content": "Explain quantum computing in one sentence"}], \
    "stream": false, \
    "temperature": 0.7, \
    "max_tokens": 100, \
    "top_p": 0.9 \
  }'

╔════════════════════════════════════════════════════════════════╗
║          Dynamo Workshop - Multi-User Configuration            ║
╠════════════════════════════════════════════════════════════════╣
║ User: nealv (UID: 1030)
║ Namespace: dynamo-nealv
╠════════════════════════════════════════════════════════════════╣
║ Your assigned ports:
║   JupyterLab:        8918
║   Frontend (Lab 1):  10030
║   Frontend (Lab 2):  11030
║   Prometheus:        19120
║   Grafana:           13030
╠════════════════════════════════════════════════════════════════╣
║ SSH Tunnel Command (run on your local machine):
║   ssh -L 8888:localhost:8918 \
║       -L 10000:localhost:10030 \
║       -L 11000:localhost:11030 \
║       nealv@<workstation-hostname>
╚════════════════════════════════════════════════════════════════╝
{"id":"chatcmpl-95f06597-6ea5-4bc0-985d-2daaf1d07d10","choices":[{"index":0,"message":{"content":"Quantum computing harnesses the principles of quantum mechanics to process information usin

## Section 5: Benchmarking with AI-Perf

### Objectives
- Install and configure AI-Perf benchmarking tool
- Run performance benchmarks against your Kubernetes deployment
- Analyze throughput, latency, and token metrics
- Compare performance across different configurations

### Metrics to Measure
- Throughput (requests/second, tokens/second)
- Latency (TTFT - Time To First Token, TPOT - Time Per Output Token, end-to-end)
- GPU utilization
- KV cache efficiency

### Benchmarking Setup
You'll run AI-Perf from your local machine against the port-forwarded service, simulating:
- Different concurrency levels (fixed concurrent requests)
- Request rate patterns (requests per second)
- Various workload characteristics

### Step 1: Install AI-Perf (if not already installed)


In [27]:
# Install AI-Perf benchmarking tool
!uv pip install aiperf -q
print("✓ AI-Perf installed")

╔════════════════════════════════════════════════════════════════╗
║          Dynamo Workshop - Multi-User Configuration            ║
╠════════════════════════════════════════════════════════════════╣
║ User: nealv (UID: 1030)
║ Namespace: dynamo-nealv
╠════════════════════════════════════════════════════════════════╣
║ Your assigned ports:
║   JupyterLab:        8918
║   Frontend (Lab 1):  10030
║   Frontend (Lab 2):  11030
║   Prometheus:        19120
║   Grafana:           13030
╠════════════════════════════════════════════════════════════════╣
║ SSH Tunnel Command (run on your local machine):
║   ssh -L 8888:localhost:8918 \
║       -L 10000:localhost:10030 \
║       -L 11000:localhost:11030 \
║       nealv@<workstation-hostname>
╚════════════════════════════════════════════════════════════════╝
✓ AI-Perf installed


### Step 2: Run Baseline Benchmark (Low Concurrency)


In [28]:
%%bash
# Run a simple benchmark with low concurrency
aiperf profile \
  --model Qwen/Qwen2.5-1.5B-Instruct \
  --url http://localhost:${USER_FRONTEND_PORT} \
  --endpoint-type chat \
  --streaming \
  --concurrency 1 \
  --request-count 100

echo ""
echo "✓ Baseline benchmark complete"

[>1u[?2048$p                                 NVIDIA AIPerf                                  
╭─── Profile Progress ───╮╭──────────────── Real-Time Metrics ─────────────────╮
│                        ││                                                    │
│                        ││                                                    │
│                        ││                                                    │
│                        ││                                                    │
│                        ││                                                    │
│  Waiting for profile   ││      No metrics available yet. Please wait...      │
│  data...               ││                                                    │
│                        ││                                                    │
│                        ││                                                    │
│                        ││                                                    │
│             



                                    NVIDIA AIPerf | LLM Metrics                                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃                             Metric ┃    avg ┃    min ┃    max ┃    p99 ┃    p90 ┃    p50 ┃   std ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│           Time to First Token (ms) │ 176.34 │ 165.89 │ 520.24 │ 184.37 │ 177.61 │ 174.57 │ 34.81 │
│          Time to Second Token (ms) │   2.08 │   0.42 │   4.11 │   2.97 │   2.73 │   2.15 │  0.55 │
│               Request Latency (ms) │ 219.65 │ 204.27 │ 637.19 │ 228.27 │ 220.06 │ 217.11 │ 42.17 │
│           Inter Token Latency (ms) │   2.91 │   2.69 │   7.80 │   3.00 │   2.92 │   2.87 │  0.49 │
│   Output Token Throughput Per User │ 347.26 │ 128.26 │ 371.69 │ 360.45 │ 356.60 │ 348.59 │ 22.71 │
│                  (tokens/sec/user) │        │        │        │        │        │      

### Step 3: Run Benchmark with Higher Concurrency


In [ ]:
%%bash
# Test with higher concurrency to stress test
aiperf profile \
  --model Qwen/Qwen2.5-1.5B-Instruct \
  --url http://localhost:${USER_FRONTEND_PORT} \
  --endpoint-type chat \
  --streaming \
  --concurrency 4 \
  --request-count 200

echo ""
echo "✓ High concurrency benchmark complete"

### Step 4: Run Benchmark with Request Rate


In [ ]:
%%bash
# Test with request rate instead of concurrency
aiperf profile \
  --model Qwen/Qwen2.5-1.5B-Instruct \
  --url http://localhost:${USER_FRONTEND_PORT} \
  --endpoint-type chat \
  --streaming \
  --request-rate 10 \
  --request-count 200

echo ""
echo "✓ Request rate benchmark complete"

### Step 5: Analyze Results

Review the benchmark outputs above. Key metrics to look for:
- **Throughput**: requests/second and tokens/second
- **TTFT (Time To First Token)**: How quickly does the first token appear?
- **TPOT (Time Per Output Token)**: Generation speed
- **End-to-end latency**: Total request time


### Cleanup

When you're done with Lab 1, clean up your deployment:


In [29]:
%%bash
# Delete the deployment
kubectl delete dynamographdeployment vllm-agg-router -n $NAMESPACE

echo ""
echo "✓ Deployment deleted"
echo "Verifying pods are terminating:"
kubectl get pods -n $NAMESPACE

dynamographdeployment.nvidia.com "vllm-agg-router" deleted from dynamo-nealv namespace

✓ Deployment deleted
Verifying pods are terminating:
NAME                                                              READY   STATUS        RESTARTS        AGE
dynamo-platform-dynamo-operator-controller-manager-64bbfc9mwkqw   2/2     Running       0               17m
dynamo-platform-etcd-0                                            1/1     Running       0               17m
dynamo-platform-nats-0                                            2/2     Running       0               17m
vllm-agg-router-frontend-6dfd867bd-6nc9n                          0/1     Completed     0               15m
vllm-agg-router-vllmdecodeworker-76b79c4bf6-xdrdl                 1/1     Terminating   1 (6m14s ago)   6m58s


**Note:** Keep your namespace and platform for Lab 2! Only delete the deployment, not the namespace.

## Troubleshooting

### Check Pod Status


In [ ]:
%%bash
# Check all pods in your namespace
kubectl get pods -n $NAMESPACE

# Describe a specific pod to see errors
# Replace <pod-name> with actual pod name from above output
# kubectl describe pod <pod-name> -n $NAMESPACE

### View Pod Logs


In [ ]:
%%bash
# View logs from a specific component
# For frontend:
kubectl logs -l component=Frontend -n $NAMESPACE --tail=50

# For worker:
kubectl logs -l component=VllmDecodeWorker -n $NAMESPACE --tail=50

### Check Deployment Status


In [ ]:
%%bash
# Check DynamoGraphDeployment status
kubectl describe dynamographdeployment vllm-agg-router -n $NAMESPACE

# Check operator logs
kubectl logs -l app.kubernetes.io/name=dynamo-operator -n $NAMESPACE --tail=50

### Check Recent Events


In [ ]:
%%bash
# View recent events in your namespace
kubectl get events -n $NAMESPACE --sort-by='.lastTimestamp' | tail -20

### Common Issues

1. **ImagePullBackOff**: Check if you have access to NGC containers. Verify image version is correct.
2. **Pods stuck in Pending**: Check if GPU resources are available: `kubectl describe pod <pod-name> -n $NAMESPACE`
3. **Model download slow**: First run takes longer due to model download. Check worker logs for progress.
4. **Port forward not working**: Make sure pods are `1/1 Ready` before forwarding. Kill existing port-forward processes: `pkill -f port-forward`

---

## Summary

### What You Learned
- ✅ How to set up a namespace-scoped Dynamo deployment on Kubernetes
- ✅ Kubernetes-based aggregated deployment architecture
- ✅ Creating and managing DynamoGraphDeployment resources
- ✅ Backend engine deployment (vLLM)
- ✅ Testing with OpenAI-compatible API
- ✅ Performance benchmarking with AI-Perf

### Key Takeaways
- Namespace-scoped operators enable safe multi-tenant deployments
- Aggregated serving is simpler to deploy and suitable for single-node models
- KV-cache routing provides intelligent load balancing across replicas
- DynamoGraphDeployment CRD simplifies complex inference deployments
- AI-Perf provides comprehensive performance insights

### Next Steps
- **(Optional)** Complete the **Monitoring Extension** (`lab1-monitoring.md`) to set up Prometheus and Grafana for observability
- In **Lab 2**, you'll explore disaggregated serving with separate prefill and decode workers, and use AIConfigurator to optimize configurations for larger models

---

## Appendix: Step-by-Step Commands

This appendix provides complete commands for each section. Use these as a reference during the lab.

**Note for MicroK8s users:** Replace `kubectl` with `microk8s kubectl` in all commands below, or set up an alias:


In [ ]:
%%python
alias kubectl='microk8s kubectl'

### A1. Environment Setup


In [ ]:
%%python
# Verify kubectl is installed and configured
kubectl version --client
kubectl cluster-info

# Set your configuration (customize with your name!)
export NAMESPACE="dynamo-yourname"  # Replace 'yourname' with your actual name
export RELEASE_VERSION="0.5.0"     # Dynamo version
export HF_TOKEN="your_hf_token"    # Your HuggingFace token
export CACHE_PATH="/data/huggingface-cache"  # Shared cache path (ask instructor)

# Create your personal namespace
kubectl create namespace ${NAMESPACE}

# Verify namespace was created
kubectl get namespace ${NAMESPACE}

# Check GPU nodes are available (optional)
kubectl get nodes -o custom-columns=NAME:.metadata.name,GPUs:.status.capacity.nvidia\\.com/gpu

### A2. Install Dynamo Platform (Namespace-Scoped)


In [ ]:
%%python
# Step 1: Check if CRDs are already installed (cluster-wide)
if kubectl get crd dynamographdeployments.nvidia.com &>/dev/null && \
   kubectl get crd dynamocomponentdeployments.nvidia.com &>/dev/null; then
    echo "✓ CRDs already installed"
else
    echo "⚠️  CRDs not found. Ask instructor to install them, or run:"
    echo "helm fetch https://helm.ngc.nvidia.com/nvidia/ai-dynamo/charts/dynamo-crds-${RELEASE_VERSION}.tgz"
    echo "helm install dynamo-crds dynamo-crds-${RELEASE_VERSION}.tgz --namespace default"
fi

# Step 2: Download Dynamo platform helm chart
helm fetch https://helm.ngc.nvidia.com/nvidia/ai-dynamo/charts/dynamo-platform-${RELEASE_VERSION}.tgz

# Step 3: Install namespace-scoped Dynamo platform
# IMPORTANT: --set dynamo-operator.namespaceRestriction.enabled=true restricts operator to this namespace
helm install dynamo-platform dynamo-platform-${RELEASE_VERSION}.tgz \
  --namespace ${NAMESPACE} \
  --set dynamo-operator.namespaceRestriction.enabled=true

# Step 4: Wait for platform pods to be ready (~2-3 minutes)
echo "Waiting for platform pods to be ready..."
kubectl wait --for=condition=ready pod \
  --all \
  --namespace ${NAMESPACE} \
  --timeout=300s

# Step 5: Verify platform is running
kubectl get pods -n ${NAMESPACE}
# You should see: dynamo-operator, etcd, and nats pods in Running state

# Step 6: Create HuggingFace token secret
kubectl create secret generic hf-token-secret \
  --from-literal=HF_TOKEN="${HF_TOKEN}" \
  --namespace ${NAMESPACE}

# Verify secret was created
kubectl get secret hf-token-secret -n ${NAMESPACE}

### A3. Deploy Your First Model

Create a deployment YAML file `agg_router.yaml`:

```yaml
# agg_router.yaml
apiVersion: nvidia.com/v1alpha1
kind: DynamoGraphDeployment
metadata:
  name: vllm-agg-router
spec:
  services:
    Frontend:
      dynamoNamespace: vllm-agg-router
      componentType: frontend
      replicas: 1
      extraPodSpec:
        mainContainer:
          image: nvcr.io/nvidia/ai-dynamo/vllm-runtime:0.5.0
      envs:
        - name: DYN_ROUTER_MODE
          value: kv
    VllmDecodeWorker:
      envFromSecret: hf-token-secret
      dynamoNamespace: vllm-agg-router
      componentType: worker
      replicas: 1
      resources:
        limits:
          gpu: "1"
      envs:
        - name: DYN_LOG
          value: "info"
      extraPodSpec:
        volumes:
        - name: local-model-cache
          hostPath:
            path: /data/huggingface-cache  # Update if instructor provides different path
            type: DirectoryOrCreate
        mainContainer:
          image: nvcr.io/nvidia/ai-dynamo/vllm-runtime:0.5.0
          volumeMounts:
          - name: local-model-cache
            mountPath: /root/.cache
          workingDir: /workspace/components/backends/vllm
          command:
            - /bin/sh
            - -c
          args:
            - python3 -m dynamo.vllm --model Qwen/Qwen2.5-1.5B-Instruct
```

Deploy the model:


In [ ]:
%%python
# Apply the deployment
kubectl apply -f agg_router.yaml --namespace ${NAMESPACE}

# Monitor deployment progress
kubectl get dynamographdeployment -n ${NAMESPACE}

# Watch pods starting up (this takes 4-6 minutes for first run)
kubectl get pods -n ${NAMESPACE} -w
# Press Ctrl+C to stop watching

# Check specific pod status
kubectl get pods -n ${NAMESPACE} | grep vllm

# View worker logs to see model loading progress
WORKER_POD=$(kubectl get pods -n ${NAMESPACE} | grep vllmdecodeworker | head -1 | awk '{print $1}')
kubectl logs ${WORKER_POD} -n ${NAMESPACE} --tail=50 --follow

### A4. Test the Deployment


In [ ]:
%%python
# Forward the frontend service port (run in a separate terminal, or add & to background)
kubectl port-forward deployment/vllm-agg-router-frontend $USER_FRONTEND_PORT:8000 -n ${NAMESPACE}

# In another terminal, test the deployment:

# Test 1: Check available models
curl http://localhost:${USER_FRONTEND_PORT}/v1/models

# Test 2: Simple non-streaming chat completion
curl http://localhost:${USER_FRONTEND_PORT}/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{
    "model": "Qwen/Qwen2.5-1.5B-Instruct",
    "messages": [{"role": "user", "content": "Hello! How are you?"}],
    "stream": false,
    "max_tokens": 50
  }'

# Test 3: Streaming chat completion
curl http://localhost:${USER_FRONTEND_PORT}/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{
    "model": "Qwen/Qwen2.5-1.5B-Instruct",
    "messages": [{"role": "user", "content": "Write a short poem about AI"}],
    "stream": true,
    "max_tokens": 100
  }'

# Test 4: With different parameters
curl http://localhost:${USER_FRONTEND_PORT}/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{
    "model": "Qwen/Qwen2.5-1.5B-Instruct",
    "messages": [{"role": "user", "content": "Explain quantum computing in one sentence"}],
    "stream": false,
    "temperature": 0.7,
    "max_tokens": 100,
    "top_p": 0.9
  }'

### A5. Benchmark with AI-Perf


In [ ]:
# Install AI-Perf (if not already installed)
pip install aiperf

# Run a simple benchmark (adjust parameters as needed)
aiperf profile \
  --model Qwen/Qwen2.5-1.5B-Instruct \
  --url http://localhost:${USER_FRONTEND_PORT} \
  --endpoint-type chat \
  --streaming \
  --concurrency 1 \
  --request-count 100

# Run with higher concurrency
aiperf profile \
  --model Qwen/Qwen2.5-1.5B-Instruct \
  --url http://localhost:${USER_FRONTEND_PORT} \
  --endpoint-type chat \
  --streaming \
  --concurrency 4 \
  --request-count 200

# Run with request rate
aiperf profile \
  --model Qwen/Qwen2.5-1.5B-Instruct \
  --url http://localhost:${USER_FRONTEND_PORT} \
  --endpoint-type chat \
  --streaming \
  --request-rate 10 \
  --request-count 200

### A6. Scale Your Deployment


In [ ]:
%%python
# Edit your agg_router.yaml and change replicas from 1 to 2
# Then reapply:
kubectl apply -f agg_router.yaml --namespace ${NAMESPACE}

# Watch the new worker come online
kubectl get pods -n ${NAMESPACE} -w

# Test that load is distributed (KV-cache routing should work)
# Run multiple requests and check logs from both workers
kubectl logs -l component=VllmDecodeWorker -n ${NAMESPACE} --tail=20

### A7. Cleanup


In [ ]:
%%python
# Delete the deployment
kubectl delete dynamographdeployment vllm-agg-router -n ${NAMESPACE}

# Verify pods are terminating
kubectl get pods -n ${NAMESPACE}

# (Optional) Keep your namespace for Lab 2
# To completely clean up (only if you're done with all labs):
# kubectl delete namespace ${NAMESPACE}

### A8. Troubleshooting


In [ ]:
%%python
# Check pod status
kubectl get pods -n ${NAMESPACE}

# Describe a pod to see errors
kubectl describe pod <pod-name> -n ${NAMESPACE}

# View logs from a specific pod
kubectl logs <pod-name> -n ${NAMESPACE}

# Check DynamoGraphDeployment status
kubectl describe dynamographdeployment vllm-agg-router -n ${NAMESPACE}

# Check operator logs
kubectl logs -l app.kubernetes.io/name=dynamo-operator -n ${NAMESPACE}

# Check if image pull is working
kubectl get events -n ${NAMESPACE} --sort-by='.lastTimestamp'